In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

['train_V2.csv', 'test_V2.csv', 'sample_submission_V2.csv']


In [2]:
from keras.models import Sequential
from keras.layers import Dense , Dropout , Flatten
from keras.optimizers import Adam 
from sklearn.model_selection import train_test_split
from keras.preprocessing.image import ImageDataGenerator
import pandas as pd
import numpy as np


Using TensorFlow backend.


In [3]:
df = pd.read_csv("../input/train_V2.csv")
df1 = pd.read_csv("../input/test_V2.csv")

print(df.shape)


(4446966, 29)


In [4]:
df.head()

,Id,groupId,matchId,assists,boosts,damageDealt,DBNOs,headshotKills,heals,killPlace,killPoints,kills,killStreaks,longestKill,matchDuration,matchType,maxPlace,numGroups,rankPoints,revives,rideDistance,roadKills,swimDistance,teamKills,vehicleDestroys,walkDistance,weaponsAcquired,winPoints,winPlacePerc
0,7f96b2f878858a,4d4b580de459be,a10357fd1a4a91,0,0,0.00,0,0,0,60,1241,0,0,0.00,1306,squad-fpp,28,26,-1,0,0.0000,0,0.00,0,0,244.80,1,1466,0.4444
1,eef90569b9d03c,684d5656442f9e,aeb375fc57110c,0,0,91.47,0,0,0,57,0,0,0,0.00,1777,squad-fpp,26,25,1484,0,0.0045,0,11.04,0,0,1434.00,5,0,0.6400
2,1eaf90ac73de72,6a4a42c3245a74,110163d8bb94ae,1,0,68.00,0,0,0,47,0,0,0,0.00,1318,duo,50,47,1491,0,0.0000,0,0.00,0,0,161.80,2,0,0.7755
3,4616d365dd2853,a930a9c79cd721,f1f1f4ef412d7e,0,0,32.90,0,0,0,75,0,0,0,0.00,1436,squad-fpp,31,30,1408,0,0.0000,0,0.00,0,0,202.70,3,0,0.1667
4,315c96c26c9aac,de04010b3458dd,6dc8ff871e21e6,0,0,100.00,0,0,0,45,0,1,1,58.53,1424,solo-fpp,97,95,1560,0,0.0000,0,0.00,0,0,49.75,2,0,0.1875


In [5]:
x_train=df.iloc[:,3:28].values
y_train=df.iloc[:,28].values
x_test=df1.iloc[:,3:28].values

In [6]:
from sklearn.preprocessing import LabelEncoder,OneHotEncoder
le=LabelEncoder()
x_train[:,12]=le.fit_transform(x_train[:,12])
x_test[:,12]=le.fit_transform(x_test[:,12])


In [7]:
print(x_train[:,12])

[15 15 2 ... 15 15 13]


In [8]:
onehot=OneHotEncoder(categorical_features=[12])
x_train=onehot.fit_transform(x_train).toarray()
x_test=onehot.fit_transform(x_test).toarray()

/opt/conda/lib/python3.6/site-packages/sklearn/preprocessing/_encoders.py:368: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)
/opt/conda/lib/python3.6/site-packages/sklearn/preprocessing/_encoders.py:390: DeprecationWarning: The 'categorical_features' keyword is deprecated in version 0.20 and will be removed in 0.22. You can use the ColumnTransformer instead.
  "use the ColumnTransformer instead.", DeprecationWarning)
/opt/conda/lib/python3.6/site-packages/sklearn/preprocessing/_encoders.py:368: FutureWarning: The handling of integer data w

In [9]:
x_train

array([[0.000e+00, 0.000e+00, 0.000e+00, ..., 2.448e+02, 1.000e+00,
        1.466e+03],
       [0.000e+00, 0.000e+00, 0.000e+00, ..., 1.434e+03, 5.000e+00,
        0.000e+00],
       [0.000e+00, 0.000e+00, 1.000e+00, ..., 1.618e+02, 2.000e+00,
        0.000e+00],
       ...,
       [0.000e+00, 0.000e+00, 0.000e+00, ..., 7.887e+02, 4.000e+00,
        0.000e+00],
       [0.000e+00, 0.000e+00, 0.000e+00, ..., 2.748e+03, 8.000e+00,
        0.000e+00],
       [0.000e+00, 0.000e+00, 0.000e+00, ..., 1.244e+03, 5.000e+00,
        0.000e+00]])

In [10]:
from sklearn.preprocessing import StandardScaler
sc=StandardScaler()
sc.fit(x_train)
sc.fit(x_test)


StandardScaler(copy=True, with_mean=True, with_std=True)

In [11]:
print(x_train.shape)
for i in range(0,40):
    x_train[:,i]=x_train[:,i]/max(x_train[:,i])
for i1 in range(0,40):
    x_test[:,i1]=x_test[:,i1]/max(x_test[:,i1])    

(4446966, 40)


In [12]:
from sklearn.decomposition import PCA
pca=PCA(n_components=10)
x_train=pca.fit_transform(x_train)
x_test=pca.fit_transform(x_test)

expvar=pca.explained_variance_ratio_
expvar


array([0.28755754, 0.19297107, 0.16650471, 0.14264905, 0.06879098,
       0.06801818, 0.04286461, 0.01111407, 0.00444137, 0.00273514])

In [13]:
x_train

array([[ 0.76573331,  0.47915954,  0.07937307, ...,  0.09897804,
         0.00384379, -0.02730405],
       [ 0.6589183 , -0.47592188,  0.00721778, ..., -0.08973063,
         0.0112509 , -0.0338613 ],
       [-0.3079626 , -0.37892598,  0.09637295, ...,  0.16264133,
        -0.03151905, -0.03091245],
       ...,
       [ 0.64777578, -0.47209458,  0.0241055 , ...,  0.11097883,
        -0.00498454, -0.00366489],
       [ 0.66298492, -0.46567492,  0.00255351, ...,  0.08722544,
        -0.00614747,  0.04318252],
       [-0.77776448, -0.36309972,  0.92746713, ..., -0.07725352,
        -0.01426065, -0.01459459]])

In [14]:
x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size=0.20, random_state=0)


In [15]:
print(x_train.shape)

(3557572, 10)


In [16]:
model=Sequential()
model.add(Dense(1024,activation='relu',input_dim=x_train.shape[1]))
model.add(Dropout(0.1))
model.add(Dense(1,activation='linear'))
model.compile(optimizer='Adam',loss='mean_squared_error',metrics=['mean_absolute_error'])
model.fit(x_train,y_train,epochs=10,batch_size=512)
model.summary()

Epoch 1/10
3557572/3557572 [==============================] - 29s 8us/step - loss: 0.0182 - mean_absolute_error: 0.0960
Epoch 2/10
3557572/3557572 [==============================] - 28s 8us/step - loss: 0.0166 - mean_absolute_error: 0.0911
Epoch 3/10
3557572/3557572 [==============================] - 28s 8us/step - loss: 0.0164 - mean_absolute_error: 0.0900
Epoch 4/10
3557572/3557572 [==============================] - 28s 8us/step - loss: 0.0163 - mean_absolute_error: 0.0896
Epoch 5/10
3557572/3557572 [==============================] - 28s 8us/step - loss: 0.0162 - mean_absolute_error: 0.0893
Epoch 6/10
3557572/3557572 [==============================] - 28s 8us/step - loss: 0.0161 - mean_absolute_error: 0.0892
Epoch 7/10
3557572/3557572 [==============================] - 29s 8us/step - loss: 0.0160 - mean_absolute_error: 0.0890
Epoch 8/10
3557572/3557572 [==============================] - 28s 8us/step - loss: 0.0160 - mean_absolute_error: 0.0889
Epoch 9/10
3557572/3557572 [============

In [17]:
predictions= model.predict(x_test)


In [20]:
predictions=predictions.tolist

In [23]:
my_submission = pd.DataFrame({'Id':df1.Id,'winPlacePerc':predictions})
my_submission.to_csv("submission.csv",index=False)
print('A submission file has been made')



A submission file has been made
